In [1]:
# Import the Earth Engine Python Package
import ee

# Initialize the Earth Engine object, using the authentication credentials.
ee.Initialize()

In [2]:
# Import the required Images and ImageCollections
sn = ee.FeatureCollection("ft:1VX1Uny1uAIYdgfuvsjLJVw1wMkhmSZq7fCQ0gowV");
modis = ee.ImageCollection("MODIS/006/MOD13Q1");
conifer_forest = ee.Image("users/mkoontz/sierra-nevada-250m-calveg-conifer-forested-pixels-by-whr-type-no-mask-full-cell");

# How many images are in the MODIS ImageCollection?
print(modis.size().getInfo());

391


In [3]:
# Display an image to make sure everything is working as expected.

from IPython.display import Image, display, HTML

thumburl = conifer_forest.getThumbUrl({
                'min':0, 
                'max':1          
            })

#print thumburl
img_thumb = Image(url=thumburl)
display(img_thumb)

In [4]:
# Define the masking function that will determine which pixels are both forested AND of high MODIS quality.

def quality_forest_mask(img):
#   date = ee.Date(img.get('system:time_start')).format("YYYYMMdd")
#   date = ee.Number.parse(date)
  
  date = ee.Image(ee.Number(img.get('system:time_start'))).rename(["date"]).divide(1000)
     
  quality_pixels = img.select(["SummaryQA"]).rename(["quality_pixel"]).eq(0)

  clean_img = img.select(["NDVI", 
        "EVI", 
        "sur_refl_b01", 
        "sur_refl_b02", 
        "sur_refl_b03", 
        "sur_refl_b07",
        "ViewZenith",
        "SolarZenith",
        "RelativeAzimuth",
        "DayOfYear",
        "SummaryQA",
        "DetailedQA"]).addBands(date).updateMask(quality_pixels).updateMask(conifer_forest).toInt32()

#   In order to export a "masked" image, fill the masked pixels with a silly number
#   All "masked" pixels will have a value of 1 (or -9999 for the commented out code)
#   I switched it to 1 in an effort to save space?

#   First, generate a dummy image with the same number of bands as the MODIS image
#   var filled_img = ee.Image([ -9999, -9999, -9999, 
#                             -9999, -9999, -9999, 
#                             -9999, -9999, -9999, 
#                             -9999, -9999, -9999])
  filled_img = ee.Image([1, 1, 1, 
                        1, 1, 1, 
                        1, 1, 1, 
                        1, 1, 1, 1]).rename(["NDVI", 
                                                "EVI", 
                                                "sur_refl_b01", 
                                                "sur_refl_b02", 
                                                "sur_refl_b03", 
                                                "sur_refl_b07",
                                                "ViewZenith",
                                                "SolarZenith",
                                                "RelativeAzimuth",
                                                "DayOfYear",
                                                "SummaryQA",
                                                "DetailedQA",
                                                "date"])

#   Where there is a mask in the clean image, replace the value of the clean image with the 
#   value from the dummy image (with all 1 (or -9999 from the commented out code))
  export_img = filled_img.where(clean_img.mask(), clean_img).clip(sn.geometry().bounds())
  export_img = ee.Image(export_img.copyProperties(clean_img, ['system:time_start']))

  return export_img;

In [5]:
# Map the masking function to the MODIS image collection

m_modis = modis.map(quality_forest_mask);
print(ee.Image(m_modis.first()).getInfo());

{'properties': {'system:index': '2000_02_18', 'system:footprint': {'coordinates': [[[-121.61056999999998, 35.159639999999975], [-117.86847999999999, 35.159639999999975], [-117.86847999999999, 40.42550000000002], [-121.61056999999998, 40.42550000000002], [-121.61056999999998, 35.159639999999975]]], 'geodesic': False, 'type': 'Polygon'}, 'system:time_start': 950832000000}, 'bands': [{'dimensions': [5, 6], 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'origin': [-122, 35], 'id': 'NDVI', 'data_type': {'max': 2147483647, 'precision': 'int', 'min': -2147483648, 'type': 'PixelType'}, 'crs': 'EPSG:4326'}, {'dimensions': [5, 6], 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'origin': [-122, 35], 'id': 'EVI', 'data_type': {'max': 2147483647, 'precision': 'int', 'min': -2147483648, 'type': 'PixelType'}, 'crs': 'EPSG:4326'}, {'dimensions': [5, 6], 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'origin': [-122, 35], 'id': 'sur_refl_b01', 'data_type': {'max': 2147483647, 'precision': 'int', 

In [6]:
# Test the first image
from IPython.display import Image, display, HTML
test_img = ee.Image(m_modis.first()).select(["EVI"]).clip(sn);

thumburl = test_img.getThumbUrl({
                'min':0, 
                'max':100,
            })

#print thumburl
img_thumb = Image(url=thumburl)
display(img_thumb)

In [9]:
# loop through all images and create a feature representing the image number and acquisition date
# Export each image upon looping

# There are 391 images
# print(m_modis.size().getInfo());

first_img = 316
num_imgs = 391

# Create a list with num_imgs empty list elements
# metadata = ee.List.sequence({
#   'start': 0,
#   'end': 390,
#   'step': 1
# })
# metadata = [None]*num_imgs
print(first_img)
# print("sn-whole-ts-modis-forest-quality-mask-%s"%str(date))
# Loop through all images, add the date to the appropriate list element, and export image
for i in range(first_img, num_imgs):
    img = ee.Image(m_modis.toList(1, i).get(0));
    date = ee.Date(img.get('system:time_start')).format("YYYYMMdd");
  
    task = ee.batch.Export.image.toDrive(**{
        'image': img, 
        'description': "sn-whole-ts-modis-forest-quality-mask-" + str(date.getInfo()), 
        'folder': 'ee/sierra-nevada-forest-quality-mask-modis-time-series', 
        'scale': 250, 
        'region': sn.geometry().getInfo()['coordinates'], 
        'crs': 'EPSG:3310'})
    task.start()

316
